In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'src', 'features')))

In [ ]:
# Utility imports
import tqdm
import pickle
import gc
from preprocessing import get_list_of_datapaths, load_data, make_train_test_datasets, split_sequence

# Math and matrix manipulation imports
import numpy as np

# Machine learning imports
import tensorflow as tf
from tensorflow import keras
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), 'Physical GPUs,', len(logical_gpus), 'Logical GPUs')
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
DATA_PATH = "../datasets/traffic/"

list_of_datapaths = get_list_of_datapaths(DATA_PATH, sort = True)
df = load_data(list_of_datapaths, [(5,8), (8, 5), (5,12), (8, 12)])
train_df, test_df = make_train_test_datasets(df, split_point=20000)

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

def mda(y_true, y_pred):
    return np.mean((np.sign(y_true[1:] - y_pred[:-1]) == np.sign(y_pred[1:] - y_pred[:-1]).astype(int)))

def calculate_metrics(y_real, y_pred):
    metrics = {
        "MAPE": mean_absolute_percentage_error(y_real, y_pred),
        "sMAPE": smape(y_real, y_pred),
        "MAE": mean_absolute_error(y_real, y_pred),
        "MDA": mda(y_real, y_pred)
    }
    return metrics

In [ ]:
X_train, y_train = split_sequence(sequence = train_df["5->8"], n_input_steps=5, n_output_steps=1)

In [ ]:
X_train

In [ ]:
# Uczenie modeli MLP
list_n_input_steps = [10, 25]
list_n_output_steps = [1]
list_targets = ['5->8', "8->5", "5->12", "8->12"]

for n_input_steps in tqdm.tqdm(list_n_input_steps):
    for n_output_steps in list_n_output_steps:
        for target in list_targets:
            X_train, y_train = prep.split_sequence(sequence = train_df[target], n_input_steps=n_input_steps, n_output_steps=n_output_steps)
            X_test, y_test = prep.split_sequence(sequence = test_df[target], n_input_steps=n_input_steps, n_output_steps=n_output_steps)

            #! Dla wielkości wyjścia m=1 | y_train = np.ravel(y_train) 
            regr = MLPRegressor(max_iter=500, hidden_layer_sizes=(100, ), random_state=0).fit(X_train, y_train)
            y_pred = regr.predict(X_test)
            #! Dla wielkości wyjścia m=1 | y_test = np.ravel(y_test)
            
            metrics = calculate_metrics(y_test, y_pred)
            metrics['y_test'] = y_test
            metrics['y_pred'] = y_pred

            file_name = f"mlp_{n_input_steps}in_{n_output_steps}out_target{target.replace('>', '')}.pkl"

            with open(f'./deeplearning_results/{file_name}', "wb") as f:
                pickle.dump(metrics, f)

In [ ]:
# Uczenie modeli BiLSTM
list_n_input_steps = [1, 10, 25]
list_n_output_steps = [1, 10, 25]
list_targets = ['5->8', "8->5", "5->12", "8->12"]

for n_input_steps in list_n_input_steps:
    for n_output_steps in list_n_output_steps:
        for target in list_targets:
            X_train, y_train = prep.split_sequence(sequence = train_df[target], n_input_steps=n_input_steps, n_output_steps=n_output_steps)
            X_test, y_test = prep.split_sequence(sequence = test_df[target], n_input_steps=n_input_steps, n_output_steps=n_output_steps)

            x_t = tf.expand_dims(X_train, axis=-1)
            y_t = tf.expand_dims(y_train, axis=-1)

            model = tf.keras.models.Sequential()
            model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=True, input_shape=x_t.shape[1::])))
            model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50)))
            model.add(tf.keras.layers.Dense(y_t.shape[1]))
            model.compile(optimizer='adam', loss='mse')
            
            history = model.fit(x_t, y_t, epochs=75, batch_size=128)
            y_p = model.predict(X_test)
            metrics = calculate_metrics(y_test, y_p)
            
            metrics['y_test'] = y_test
            metrics['y_pred'] = y_p

            file_name = f"bidirectional_lstm_{n_input_steps}in_{n_output_steps}out_target{target.replace('>', '')}.pkl"

            with open(f'./deeplearning_results/{file_name}', "wb") as f:
                pickle.dump(metrics, f)

            del model
            keras.backend.clear_session()
            gc.collect()